In [42]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import jdatetime as jd


Read data






In [43]:
input_data=pd.read_csv("weather.csv", header=4)
#input_data
#input_data.info()
#input_data.describe()

Pre Processing

In [44]:
def auto_data_type(df):
    for col in df.select_dtypes(include=[np.number]).columns:
        min_value = df[col].min()
        max_value = df[col].max()

        if pd.api.types.is_float_dtype(df[col]):
            df[col] = df[col].astype(np.float32)
        elif min_value >= 0:
            if max_value <= 255:
                df[col] = df[col].astype(np.uint8)
            elif max_value <= 65535:
                df[col] = df[col].astype(np.uint16)
            elif max_value <= 4294967295:
                df[col] = df[col].astype(np.uint32)
            else:
                df[col] = df[col].astype(np.uint64)
        else:
            if -128 <= min_value and max_value <= 127:
                df[col] = df[col].astype(np.int8)
            elif -32768 <= min_value and max_value <= 32767:
                df[col] = df[col].astype(np.int16)
            elif -2147483648 <= min_value and max_value <= 2147483647:
                df[col] = df[col].astype(np.int32)
            else:
                df[col] = df[col].astype(np.int64)
    return df


reduced_data=auto_data_type(input_data)
reduced_data["sunrise (iso8601)"] = pd.to_datetime(reduced_data["sunrise (iso8601)"]).dt.strftime("%H:%M")
reduced_data["sunset (iso8601)"] = pd.to_datetime(reduced_data["sunset (iso8601)"]).dt.strftime("%H:%M")

reduced_data["daylight_duration (s)"]=(reduced_data["daylight_duration (s)"]/60).astype(np.uint16)





Remove out of range data

In [45]:
numeric_cols = [
    "temperature_2m_mean (°C)",
    "temperature_2m_max (°C)",
    "temperature_2m_min (°C)",
    "precipitation_sum (mm)",
    "rain_sum (mm)",
    "snowfall_sum (cm)"
]

cols = ['precipitation_sum (mm)', 'rain_sum (mm)', 'snowfall_sum (cm)']

for col in cols:
    mean = reduced_data[col].mean()
    std = reduced_data[col].std()
    reduced_data[f"{col}_is_out_of_range"] = (reduced_data[col] < mean - 3 * std) | (reduced_data[col] > mean + 3 * std)

out_of_range_cols = [
    'precipitation_sum (mm)_is_out_of_range',
    'rain_sum (mm)_is_out_of_range',
    'snowfall_sum (cm)_is_out_of_range'
]

condition = (
    (reduced_data[out_of_range_cols[0]] == False) &
    (reduced_data[out_of_range_cols[1]] == False) &
    (reduced_data[out_of_range_cols[2]] == False)
)

cleaned_data = reduced_data[condition]

print(f"Total rows with out_of_ranges: {len(reduced_data) - len(cleaned_data)}")
print(f"Rows remaining: {len(cleaned_data)}")



# plt.boxplot(
#     reduced_data.loc[reduced_data["location_id"]==1, "precipitation_sum (mm)"],
#     labels=["Location 0"]
# )
# plt.show()


Total rows with out_of_ranges: 37
Rows remaining: 1061


Remove duplicated

In [46]:
cleaned_data.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
1091    False
1094    False
1095    False
1096    False
1097    False
Length: 1061, dtype: bool

In [47]:
reduced_data["time"] = pd.to_datetime(reduced_data["time"])

def seasons_jalali(date):
    jdate = jd.date.fromgregorian(date=date)
    if jdate.month in [1, 2, 3]:
        return 'spring'
    elif jdate.month in [4, 5, 6]:
        return 'summer'
    elif jdate.month in [7, 8, 9]:
        return 'fall'
    else:
        return 'winter'

seasons = reduced_data["time"].map(seasons_jalali)

fall = reduced_data[seasons == 'fall']
winter = reduced_data[seasons == 'winter']
spring = reduced_data[seasons == 'spring']
summer = reduced_data[seasons == 'summer']
#print(len(fall),len(winter),len(spring),len(summer))

In [48]:
#reduced_data.info()